In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Large_B-cell_Lymphoma"
cohort = "GSE142494"

# Input paths
in_trait_dir = "../../input/GEO/Large_B-cell_Lymphoma"
in_cohort_dir = "../../input/GEO/Large_B-cell_Lymphoma/GSE142494"

# Output paths
out_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/GSE142494.csv"
out_gene_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE142494.csv"
out_clinical_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE142494.csv"
json_path = "../../output/preprocess/Large_B-cell_Lymphoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A dichotomy of gene regulatory associations during the activated B-cell to plasmablast transition"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: memory B cells', 'cell type: total B cells'], 1: ['differentiation: Day3', 'differentiation: Day4', 'differentiation: Day5', 'differentiation: Day6', 'differentiation: Day0'], 2: ['culture-conditions: At day-3 cells were detached from the CD40L L-cell layer and reseeded at 1 X 10^5/ml in media supplemented with IL-2 (20 U/ml) and IL-21 (50 ng/ml).', 'culture-conditions: B cells were cultured at 2.5 X 10^5/ml with IL-2 (20 U/ml), IL-21 (50 ng/ml), F(ab’)2 goat anti-human IgM & IgG (10 µg/ml) on γ-irradiated CD40L expressing L cells (6.25 X 10^4/well).', 'culture-conditions: At day-3 cells were detached from the CD40L L-cell layer and reseeded at 1 X 10^5/ml in media supple

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os

# 1. Gene Expression Data Availability
# Based on the series summary and overall design, this appears to be a gene expression dataset
# studying B-cell differentiation and gene regulation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Large_B-cell_Lymphoma):
# Looking at the sample characteristics, no explicit "Lymphoma" values are present
# The keys in the dictionary are:
# 0: 'cell type' - memory B cells, total B cells
# 1: 'differentiation' - Day0 through Day6
# 2: 'culture-conditions' - various treatment conditions
# 
# This dataset appears to be about normal B-cell differentiation, not lymphoma
trait_row = None

# Age data is not available in the sample characteristics
age_row = None

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Since trait data is not available, we'll define a placeholder function
def convert_trait(value):
    return None

# Similarly for age and gender
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Conduct initial filtering on the usability of the dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as trait_row is None, indicating clinical data for the trait is not available


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"A dichotomy of gene regulatory associations during the activated B-cell to plasmablast transition"
Line 1: !Series_geo_accession	"GSE142494"
Line 2: !Series_status	"Public on Aug 24 2020"
Line 3: !Series_submission_date	"Dec 21 2019"
Line 4: !Series_last_update_date	"Sep 21 2020"
Line 5: !Series_pubmed_id	"32843533"
Line 6: !Series_summary	"This SuperSeries is composed of the SubSeries listed below."
Line 7: !Series_overall_design	"Refer to individual Series"
Line 8: !Series_type	"Expression profiling by array"
Line 9: !Series_type	"Genome binding/occupancy profiling by high throughput sequencing"
Found table marker at line 68
First few lines after marker:
"ID_REF"	"GSM4230397"	"GSM4230398"	"GSM4230399"	"GSM4230400"	"GSM4230401"	"GSM4230402"	"GSM4230403"	"GSM4230404"	"GSM4230405"	"GSM4230406"	"GSM4230407"	"GSM4230408"	"GSM4230409"	"GSM4230410"	"GSM4230411"	"GSM4230412"	"GSM4230413"	"GSM4230414"	"GSM4230415"	"GSM4230416"	"GSM4230

Successfully extracted gene data with 47323 rows
First 20 gene IDs:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# The IDs start with "ILMN_" which indicates these are Illumina probe IDs
# These are not standard human gene symbols but probe identifiers from an Illumina microarray platform
# They need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 3029519 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene ID mapping
# From the preview, we can see we need to map from 'ID' (probe identifiers) to 'Symbol' (gene symbols)
print("\nCreating gene mapping from probe IDs to gene symbols...")

# 2. Extract mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print(f"Created mapping with {len(mapping_df)} rows")
print("First few rows of mapping data:")
print(preview_df(mapping_df))

# Check for mapping coverage
probe_ids_in_expr = set(gene_data.index)
probe_ids_in_map = set(mapping_df['ID'])
overlap = probe_ids_in_expr.intersection(probe_ids_in_map)
print(f"\nProbes in expression data: {len(probe_ids_in_expr)}")
print(f"Probes in mapping data: {len(probe_ids_in_map)}")
print(f"Overlap (probes that can be mapped): {len(overlap)}")
print(f"Mapping coverage: {len(overlap)/len(probe_ids_in_expr):.2%}")

# 3. Apply the mapping to convert probe-level data to gene expression data
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted to gene expression data with {len(gene_data)} genes")

# Normalize gene symbols to handle synonyms
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(gene_data)} unique genes")

# Preview the resulting gene expression data
print("\nPreview of gene expression data:")
print(preview_df(gene_data))

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")



Creating gene mapping from probe IDs to gene symbols...
Created mapping with 44837 rows
First few rows of mapping data:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}

Probes in expression data: 47323
Probes in mapping data: 44837
Overlap (probes that can be mapped): 44053
Mapping coverage: 93.09%

Converting probe-level measurements to gene expression data...


Converted to gene expression data with 21464 genes

Normalizing gene symbols...
After normalization: 20259 unique genes

Preview of gene expression data:
{'GSM4230397': [16.650590606, 8.216769318, 24.619701452, 8.020100477, 7.968417558], 'GSM4230398': [16.303410088, 8.29990581, 24.672474215, 8.001034902, 7.972244982], 'GSM4230399': [16.474239626, 8.293178378, 24.604444522, 7.998611331, 8.06327577], 'GSM4230400': [16.404515596, 8.244805096, 24.583375097, 7.983330304, 8.091351085], 'GSM4230401': [16.516765953, 8.200109698, 24.563564699, 7.981676863, 7.967531026], 'GSM4230402': [16.549311276, 8.206625807, 24.44487138, 7.986913154, 7.920091144], 'GSM4230403': [16.624553708, 8.172420111, 24.692044333, 8.00338877, 8.152064676], 'GSM4230404': [16.347913586, 8.247254959, 24.724101554999997, 8.054338462, 7.987656143], 'GSM4230405': [16.470867104, 8.112620089, 24.460172373, 8.039134877, 8.019603193], 'GSM4230406': [16.665684482, 8.273386579, 24.496097482, 8.029102956, 7.995906883], 'GSM4230407':


Gene expression data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE142494.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Gene data was already normalized and saved in the previous step
print("\nChecking normalized gene data availability...")
try:
    # Load the already normalized gene data from the correct path
    if os.path.exists(out_gene_data_file):
        print(f"Loading normalized gene data from {out_gene_data_file}...")
        gene_data = pd.read_csv(out_gene_data_file, index_col=0)
        print(f"Loaded gene data with shape: {gene_data.shape}")
        print(f"Sample of gene symbols: {gene_data.index[:5].tolist()}")
        is_gene_available = True
    else:
        print(f"ERROR: Normalized gene data file not found at {out_gene_data_file}")
        is_gene_available = False
except Exception as e:
    print(f"Error loading normalized gene data: {str(e)}")
    is_gene_available = False

# 2. Check clinical data availability
print("\nChecking clinical data availability...")
# From previous steps, we determined that trait_row was None
trait_row = None
is_trait_available = trait_row is not None

if is_trait_available:
    print("Clinical data is available for this cohort.")
    # This block won't execute since trait_row is None
else:
    print("No clinical data available for this cohort. Cannot proceed with linking.")

# 3. Since clinical data is not available, we can't create proper linked data
# But we'll use the gene data for diagnostic purposes
linked_data = gene_data.T if is_gene_available else pd.DataFrame()
print(f"Created diagnostic data frame with shape: {linked_data.shape}")

# 4. Since we can't evaluate trait bias without trait data, mark as biased
is_biased = True

# 5. Validate and save cohort information
print("\nPerforming final validation...")
note = "Dataset contains gene expression data but lacks clinical information for Large_B-cell_Lymphoma. " \
       "The samples appear to be B-cells at different stages of differentiation, not lymphoma cases."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Report final status - we don't save the linked data because it's not usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if not is_usable:
    print(f"Reason: {note}")
else:
    # This block won't execute since is_usable should be False
    print(f"Saving linked data to {out_data_file}")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)


Checking normalized gene data availability...
Loading normalized gene data from ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE142494.csv...
Loaded gene data with shape: (20259, 63)
Sample of gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1']

Checking clinical data availability...
No clinical data available for this cohort. Cannot proceed with linking.
Created diagnostic data frame with shape: (63, 20259)

Performing final validation...

Dataset usability for Large_B-cell_Lymphoma association studies: False
Reason: Dataset contains gene expression data but lacks clinical information for Large_B-cell_Lymphoma. The samples appear to be B-cells at different stages of differentiation, not lymphoma cases.
